In [6]:
import cv2
import os
import numpy as np
from google.colab.patches import cv2_imshow

In [ ]:
!unzip "/content/drive/MyDrive/MiddEval3-data-F.zip"
!unzip "/content/drive/MyDrive/MiddEval3-GTy-F.zip"
train = "/content/MiddEval3/trainingF"

In [ ]:
images = os.listdir(train)
images.sort()
images = images[1:]
images

In [32]:
mean_mae = []
mean_rmse = []
z = 1

for i in images:
    
    img_left = cv2.imread(train + "/" + i + "/" + 'im0.png')
    img_right = cv2.imread(train + "/" + i + "/" + 'im1.png')
    
    #Gaussian filter to smoothen images
    img_left = cv2.GaussianBlur(img_left, (7, 7), 1)
    img_right = cv2.GaussianBlur(img_right, (7, 7), 1)

    # Convert the images to grayscale
    gray_left = cv2.cvtColor(img_left, cv2.COLOR_BGR2GRAY)
    gray_right = cv2.cvtColor(img_right, cv2.COLOR_BGR2GRAY)

    # Find the keypoints and descriptors using SIFT
    sift = cv2.xfeatures2d.SIFT_create()
    kp_left, des_left = sift.detectAndCompute(gray_left, None)
    kp_right, des_right = sift.detectAndCompute(gray_right, None)

    # Find the matches between the descriptors
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des_left, des_right, k=2)

    # Apply ratio test to find good matches
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    # Find the coordinates of the matched keypoints
    left_pts = np.float32([kp_left[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    right_pts = np.float32([kp_right[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    # Estimate the fundamental matrix
    F, mask = cv2.findFundamentalMat(left_pts, right_pts, cv2.FM_RANSAC)

    # Rectify the images
    ret, H_left, H_right = cv2.stereoRectifyUncalibrated(left_pts, right_pts, F, img_left.shape[:2])

    # Compute the disparity map
    window_size = 5
    min_disp = 0
    max_disp = 64
    num_disp = max_disp - min_disp
    stereo = cv2.StereoSGBM_create(minDisparity=min_disp,
                                numDisparities=num_disp,
                                blockSize=window_size,
                                uniquenessRatio=10,
                                speckleWindowSize=100,
                                speckleRange=32,
                                disp12MaxDiff=5,
                                P1=8*3*window_size**2,
                                P2=32*3*window_size**2)
    gray_left_rect = cv2.warpPerspective(gray_left, H_left, img_left.shape[:2][::-1])
    gray_right_rect = cv2.warpPerspective(gray_right, H_right, img_right.shape[:2][::-1])
    disparity = stereo.compute(gray_left_rect, gray_right_rect)

    # Display the depth map
    min_disp = disparity.min()
    max_disp = disparity.max()
    disp_normalized = (disparity - min_disp) / (max_disp - min_disp)
    depth_map = cv2.applyColorMap((disp_normalized * 255).astype(np.uint8), cv2.COLORMAP_JET)
    cv2.imwrite(f'/content/drive/MyDrive/project_output/disparity_{i}.png',depth_map)

    gt_depth_map = cv2.imread(train + "/" + i + "/" + 'disp0GTy.pfm', cv2.IMREAD_GRAYSCALE).astype(np.float32) / 255.0
    computed_depth_map = cv2.imread(f'/content/drive/MyDrive/project_output/disparity_{i}.png', cv2.IMREAD_GRAYSCALE).astype(np.float32) / 255.0
    # depth_map = depth_map/255.0
    # Compute the absolute difference between the two depth maps
    # print(gt_depth_map.shape, depth_map.shape)
    abs_diff = cv2.absdiff(gt_depth_map, computed_depth_map)

    # Compute the mean absolute error (MAE) and the root mean squared error (RMSE)
    mae = np.mean(abs_diff)
    rmse = np.sqrt(np.mean(np.square(abs_diff)))

    mean_mae.append(mae)
    mean_rmse.append(rmse)

    # Print the results
    print(f'Mean Absolute Error (MAE) for img {z}:', mae)
    print(f'Root Mean Squared Error (RMSE) img {z}:', rmse)
    print("--------------------------------------------------------------------------------------------")
    print()
    z+=1
    

Mean Absolute Error (MAE) for img 1: 0.18498603
Root Mean Squared Error (RMSE) img 1: 0.29448724
--------------------------------------------------------------------------------------------

Mean Absolute Error (MAE) for img 2: 0.16449785
Root Mean Squared Error (RMSE) img 2: 0.26953125
--------------------------------------------------------------------------------------------

Mean Absolute Error (MAE) for img 3: 0.17622311
Root Mean Squared Error (RMSE) img 3: 0.28982896
--------------------------------------------------------------------------------------------

Mean Absolute Error (MAE) for img 4: 0.169583
Root Mean Squared Error (RMSE) img 4: 0.2816584
--------------------------------------------------------------------------------------------

Mean Absolute Error (MAE) for img 5: 0.27715164
Root Mean Squared Error (RMSE) img 5: 0.3892926
--------------------------------------------------------------------------------------------

Mean Absolute Error (MAE) for img 6: 0.21273646
R

In [33]:
print(f"Mean MAE on train set: {np.mean(mean_mae)}") 
print(f"Mean RMSE on train set: {np.mean(mean_rmse)}") 


Mean MAE on train set: 0.23706090450286865
Mean RMSE on train set: 0.34546616673469543
